얼굴인식 기반 참석자 마스크 착용 경보 시스템 ( 2019-12443 김성경 )

라즈베리파이 코드 설명 : 카메라 모듈을 활용하여 실시간으로 영상을 받아서 프레임별 이미지를 통해 얼굴을 인식함으로써, 얼굴이 인식된다면, 마스크를 착용하지 않은 것이므로 경보를 주었다. 


In [ ]:
import numpy as np
import cv2
import os
import tensorflow.keras
from PIL import Image, ImageOps
xml = 'haarcascade_frontalface_default.xml' # haarcascade 모델 활용
face_cascade = cv2.CascadeClassifier(xml)
f = "face.mp4"

cap = cv2.VideoCapture(0) # 라즈베리파이 카메라 모듈을 카메라로 사용

#cap.set(3,640) # 너비

#cap.set(4,480) # 높이
#model = tensorflow.keras.models.load_model('keras_model.h5',compile=False)

def classify(image): ###################################### Google Teachable Machine 을 적용하는 코드 
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

    # Load the model
    model = tensorflow.keras.models.load_model('keras_model.h5',compile=False)
    #print(model)

    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    #turn the image into a numpy array
    image_array = np.asarray(image)
    # display the resized image
    #image.show()
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    print(prediction)
    prediction=prediction[0]
    if prediction[0]>=prediction[1] and prediction[0]>=prediction[2]: # "얼굴 없음" 으로 판단할 가능성이 가장 높은 경우
      print("no face")
      return 0
    if prediction[1]>=prediction[2] and prediction[1]>=prediction[0]:# "마스크 착용 안한 얼굴" 로 판단할 가능성이 가장 높은 경우
      print("pure face")
      return 1
    if prediction[2]>=prediction[1] and prediction[2]>=prediction[0]:# "마스크 착용한 얼굴" 로 판단할 가능성이 가장 높은 경우
      print("mask face")
      return 2
  
# openCV 부분의 원리 : 
'''
구체적으로는 얼굴인식이 각 픽셀마다 이루어진다. 
'''
framecnt = 0                  ############################## 이 코드를 참고해서 코딩함 : https://github.com/kimjinho1/Real-time-face-recognition-and-mosaic-using-deep-learning/blob/master/2.%20Real-time_face_detection.ipynb
while(True):  // 출처 
    framecnt+=1
    print("onframe"+str(framecnt),end="")
    ret, frame = cap.read()
    
    if framecnt%10==0:

        frame = cv2.flip(frame, 1) # 좌우 대칭
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray)
        print("Number of faces detected: " + str(len(faces)))
        retp = 0
        if len(faces):
            for (x,y,w,h) in faces: #얼굴인식 진행 
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2) 
                if w*h>=5000:# 얼굴 부분의 넓이가 5000픽셀제곱 이상이 될 정도로 사용자가 가까워야 감지 시작 
                    retp+=1
        print("Number of REAL faces detected: " + str(retp))
        if retp>=1: # 진짜 얼굴이 하나 이상이 인식되면 마스크를 착용한 것이 아니므로 경보를 울려준다. (Google TTS API 활용)
            file = "helloEN.mp3"
            os.system("mpg123 " + file)

        cv2.imshow('result', frame)
        #resu=classify(frame)  # 만약 Teachable Machine 까지 추후 성공적으로 동작할 경우, 위 네 줄 대신  이와 같이 코딩을 진행할 수 있다. 
        #print(resu)

        #if retp>=1 or resu==1: # Teachable MAchine이 얼굴을 인식한 경우에도 경보를 울릴 수 있도록 한다. 
        #    file = "helloEN.mp3"
        #    os.system("mpg123 " + file)

        #cv2.imshow('result', frame)
       
            
            

        k = cv2.waitKey(30) & 0xff
        if k == 27: # Esc 키를 누르면 종료
            break

cap.release()
cv2.destroyAllWindows()
